# BentoML Example:  Keras Text Classification

[BentoML](http://bentoml.ai) is an open source platform for machine learning model serving and deployment. 

This notebook demonstrates how to use BentoML to turn a Keras model into a docker image containing a REST API server serving this model, how to use your ML service built with BentoML as a CLI tool, and how to distribute it a pypi package.

This notebook is built based on Keras's IMDB LSTM tutorial [here](https://github.com/keras-team/keras/blob/master/examples/imdb_lstm.py).

![Impression](https://www.google-analytics.com/collect?v=1&tid=UA-112879361-3&cid=555&t=event&ec=keras&ea=keras-text-classification&dt=keras-text-classification)

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
!pip install bentoml
!pip install tensorflow==1.14.0
!pip install numpy

In [2]:
from __future__ import absolute_import, division, print_function

import numpy as np
import tensorflow as tf
print("Tensorflow Version: %s" % tf.__version__)

from tensorflow import keras
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.datasets import imdb

import bentoml
print("BentoML Version: %s" % bentoml.__version__)

Tensorflow Version: 1.14.0
BentoML Version: 0.6.2+5.g3cbab90


In [3]:
max_features = 1000
maxlen = 80 # cut texts after this number of words (among top max_features most common words)
batch_size = 300
index_from=3 # word index offset

# Prepare Dataset
Download the IMDB dataset

In [4]:
# A dictionary mapping words to an integer index
imdb.load_data(num_words=max_features)
word_index = imdb.get_word_index()

# The first indices are reserved
word_index = {k:(v+index_from) for k,v in word_index.items()} 
word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<UNK>"] = 2  # unknown

# Use decode_review to look at original review text in training/testing data
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
def decode_review(encoded_text):
    return ' '.join([reverse_word_index.get(i, '?') for i in encoded_text])

In [5]:
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features, index_from=index_from)

In [6]:
x_train = sequence.pad_sequences(x_train,
                                 value=word_index["<PAD>"],
                                 padding='post',
                                 maxlen=maxlen)

x_test = sequence.pad_sequences(x_test,
                                value=word_index["<PAD>"],
                                padding='post',
                                maxlen=maxlen)

# Model Training & Evaluation

In [7]:
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

model.summary()

W0211 17:23:02.977754 4690394560 deprecation.py:506] From /usr/local/anaconda3/envs/dev-py3/lib/python3.7/site-packages/tensorflow/python/keras/initializers.py:119: calling RandomUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0211 17:23:02.995445 4690394560 deprecation.py:506] From /usr/local/anaconda3/envs/dev-py3/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 128)         128000    
_________________________________________________________________
lstm (LSTM)                  (None, 128)               131584    
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 259,713
Trainable params: 259,713
Non-trainable params: 0
_________________________________________________________________


In [8]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=1, # for demo purpose :P
          validation_data=(x_test, y_test))

W0211 17:23:03.373009 4690394560 deprecation.py:323] From /usr/local/anaconda3/envs/dev-py3/lib/python3.7/site-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 25000 samples, validate on 25000 samples
25000/25000 [==============================] - 35s 1ms/sample - loss: 0.5795 - acc: 0.6965 - val_loss: 0.4436 - val_acc: 0.8020


In [9]:
score, acc = model.evaluate(x_test, y_test,
                            batch_size=batch_size)

print('Test score:', score)
print('Test accuracy:', acc)

25000/25000 [==============================] - 7s 281us/sample - loss: 0.4436 - acc: 0.8020
Test score: 0.4436431996822357
Test accuracy: 0.80204


## Define BentoService for model serving

In [41]:
%%writefile keras_text_classification_service.py
import pandas as pd
import numpy as np
from tensorflow import keras
from tensorflow.keras.preprocessing import sequence, text
from bentoml import api, env, BentoService, artifacts
from bentoml.artifact import KerasModelArtifact, PickleArtifact
from bentoml.handlers import JsonHandler

max_features = 1000

@artifacts([
    KerasModelArtifact('model'),
    PickleArtifact('word_index')
])
@env(pip_dependencies=['tensorflow==1.14.0', 'numpy', 'pandas'])
class KerasTextClassificationService(BentoService):
   
    def word_to_index(self, word):
        if word in self.artifacts.word_index and self.artifacts.word_index[word] <= max_features:
            return self.artifacts.word_index[word]
        else:
            return self.artifacts.word_index["<UNK>"]
    
    def preprocessing(self, text_str):
        sequence = text.text_to_word_sequence(text_str)
        return list(map(self.word_to_index, sequence))
    
    @api(JsonHandler)
    def predict(self, parsed_json):
        if type(parsed_json) == list:
            input_data = list(map(self.preprocessing, parsed_json))
        else: # expecting type(parsed_json) == dict:
            input_data = [self.preprocessing(parsed_json['text'])]

        input_data = sequence.pad_sequences(input_data,
                                            value=self.artifacts.word_index["<PAD>"],
                                            padding='post',
                                            maxlen=80)

        return self.artifacts.model.predict_classes(input_data)

Overwriting keras_text_classification_service.py


## Save BentoService to file archive

In [ ]:
# 1) import the custom BentoService defined above
from keras_text_classification_service import KerasTextClassificationService

# 2) `pack` it with required artifacts
bento_svc = KerasTextClassificationService()
bento_svc.pack('model', model)
bento_svc.pack('word_index', word_index)

# 3) save your BentoSerivce
saved_path = bento_svc.save()

### Test packed BentoML service

In [12]:
bento_svc.predict({ 'text': 'bad worst terrible' })

array([[0]], dtype=int32)

In [13]:
bento_svc.predict(['the best movie I have ever seen', 'This is a bad movie'])

array([[1],
       [0]], dtype=int32)

# Load BentoML Service from archive

In [14]:
import bentoml

loaded_bento_svc = bentoml.load(saved_path)

[2019-10-14 17:11:11,437] WARNING - Module `text_classification_service` already loaded, using existing imported module.


W1014 17:11:11.451318 4541836608 deprecation_wrapper.py:119] From /Users/chaoyuyang/anaconda3/envs/bentoml-dev/lib/python3.7/site-packages/bentoml/artifact/keras_model_artifact.py:104: The name tf.keras.backend.set_session is deprecated. Please use tf.compat.v1.keras.backend.set_session instead.

W1014 17:11:11.467543 4541836608 deprecation.py:506] From /Users/chaoyuyang/anaconda3/envs/bentoml-dev/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:97: calling GlorotUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W1014 17:11:11.468544 4541836608 deprecation.py:506] From /Users/chaoyuyang/anaconda3/envs/bentoml-dev/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:97: calling Orthogonal.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed 

In [15]:
loaded_bento_svc.predict({ "text": "the best movie I have ever seen" })

array([[1]], dtype=int32)

In [16]:
loaded_bento_svc.predict(['the best movie I have ever seen', 'This is a bad movie'])

array([[1],
       [1]], dtype=int32)

## Using BentoService with BentoML CLI tool

**We can use `bentoml get` command to retrieve BentoService basic information**

In [14]:
!bentoml get KerasTextClassificationService

BENTO_SERVICE                                         AGE                         APIS                  ARTIFACTS
KerasTextClassificationService:20200211173435_F31009  5 minutes and 58.2 seconds  predict<JsonHandler>  model<KerasModelArtifact>, word_index<PickleArtifact>
KerasTextClassificationService:20200206112311_4FA99B  5 days and 6 hours          predict<JsonHandler>  model<KerasModelArtifact>, word_index<PickleArtifact>


When provide `version` info, `bentoml get` will retrieve additional metadata

In [15]:
!bentoml get KerasTextClassificationService:20200211173435_F31009

{
  "name": "KerasTextClassificationService",
  "version": "20200211173435_F31009",
  "uri": {
    "type": "LOCAL",
    "uri": "/Users/bozhaoyu/bentoml/repository/KerasTextClassificationService/20200211173435_F31009"
  },
  "bentoServiceMetadata": {
    "name": "KerasTextClassificationService",
    "version": "20200211173435_F31009",
    "createdAt": "2020-02-12T01:35:00.448136Z",
    "env": {
      "condaEnv": "name: bentoml-KerasTextClassificationService\nchannels:\n- defaults\ndependencies:\n- python=3.7.3\n- pip\n",
      "pipDependencies": "bentoml==0.6.2\ntensorflow\nnumpy\npandas",
      "pythonVersion": "3.7.3"
    },
    "artifacts": [
      {
        "name": "model",
        "artifactType": "KerasModelArtifact"
      },
      {
        "name": "word_index",
        "artifactType": "PickleArtifact"
      }
    ],
    "apis": [
      {
        "name": "predict",
        "handlerType": "JsonHandler",
        "docs": "BentoService API"
      }
    ]
  }
}


`bentoml run` provides a quick way to get prediction result

In [16]:
!bentoml run KerasTextClassificationService:20200211173435_F31009 predict --input='{"text": "bad movie"}'

[2020-02-11 17:41:39,097] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will be default to same version as last PyPI release at version: 0.6.2. You can also force bentoml to use a specific version for deploying your BentoService bundle, by setting the config 'core/bentoml_deploy_version' to a pinned version or your custom BentoML on github, e.g.:'bentoml_deploy_version = git+https://github.com/{username}/bentoml.git@{branch}'
[2020-02-11 17:41:39,109] WARNING - Saved BentoService bundle version mismatch: loading BentoServie bundle create with BentoML version 0.6.2,  but loading from BentoML version 0.6.2+5.g3cbab90
[2020-02-11 17:41:41,532] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will be

#### Run REST API server locally

In [17]:
!bentoml serve KerasTextClassificationService:20200211173435_F31009

[2020-02-11 17:42:06,310] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will be default to same version as last PyPI release at version: 0.6.2. You can also force bentoml to use a specific version for deploying your BentoService bundle, by setting the config 'core/bentoml_deploy_version' to a pinned version or your custom BentoML on github, e.g.:'bentoml_deploy_version = git+https://github.com/{username}/bentoml.git@{branch}'
[2020-02-11 17:42:06,322] WARNING - Saved BentoService bundle version mismatch: loading BentoServie bundle create with BentoML version 0.6.2,  but loading from BentoML version 0.6.2+5.g3cbab90
[2020-02-11 17:42:08,514] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will be

### Send prediction request to REST API server

*Run the following command in terminal to make a HTTP request to the API server*
```bash
curl -i \
--header "Content-Type: application/json" \
--request POST \
--data '{"text": "best movie ever"}' \
localhost:5000/predict
```

# "pip install" a BentoML archive

BentoML user can directly pip install saved BentoML archive with `pip install $SAVED_PATH`,  and use it as a regular python package.

In [18]:
!pip install {saved_path}

Processing /Users/bozhaoyu/bentoml/repository/KerasTextClassificationService/20200211173435_F31009


  Created wheel for KerasTextClassificationService: filename=KerasTextClassificationService-20200211173435_F31009-py3-none-any.whl size=3762773 sha256=70100f150385c4c0f6c68ee496c5fc93ccf8dac5c9604f9df4e82567b5a71851
  Stored in directory: /private/var/folders/kn/xnc9k74x03567n1mx2tfqnpr0000gn/T/pip-ephem-wheel-cache-272d7tjf/wheels/4e/70/05/ac267d80973ce0e42544c4b38c72cde7413226574f2d18ef65
Successfully built KerasTextClassificationService


In [19]:
import KerasTextClassificationService

installed_svc = KerasTextClassificationService.load()

W0211 17:50:47.581211 4690394560 deprecation_wrapper.py:119] From /Users/bozhaoyu/src/bento/bentoml/artifact/keras_model_artifact.py:114: The name tf.keras.backend.set_session is deprecated. Please use tf.compat.v1.keras.backend.set_session instead.

W0211 17:50:47.602521 4690394560 deprecation.py:506] From /usr/local/anaconda3/envs/dev-py3/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:97: calling GlorotUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0211 17:50:47.603494 4690394560 deprecation.py:506] From /usr/local/anaconda3/envs/dev-py3/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:97: calling Orthogonal.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer inst

In [20]:
installed_svc.predict({ 'text': 'the best movie I have ever seen' })

array([[1]], dtype=int32)

In [21]:
installed_svc.predict({ 'text': 'This is a bad movie' })

array([[0]], dtype=int32)

#### Additional CLI access from PyPI package

`pip install $SAVED_PATH` also installs a CLI tool for accessing the BentoML service

In [22]:
!KerasTextClassificationService --help

Usage: KerasTextClassificationService [OPTIONS] COMMAND [ARGS]...

  BentoML CLI tool

Options:
  --version  Show the version and exit.
  --help     Show this message and exit.

Commands:
  info            List APIs
  open-api-spec   Display OpenAPI/Swagger JSON specs
  run             Run API function
  serve           Start local rest server
  serve-gunicorn  Start local gunicorn server


### Print model service information:

In [23]:
!KerasTextClassificationService info

[2020-02-11 17:51:21,509] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will be default to same version as last PyPI release at version: 0.6.2. You can also force bentoml to use a specific version for deploying your BentoService bundle, by setting the config 'core/bentoml_deploy_version' to a pinned version or your custom BentoML on github, e.g.:'bentoml_deploy_version = git+https://github.com/{username}/bentoml.git@{branch}'
[2020-02-11 17:51:21,518] WARNING - Saved BentoService bundle version mismatch: loading BentoServie bundle create with BentoML version 0.6.2,  but loading from BentoML version 0.6.2+5.g3cbab90
{
  "name": "KerasTextClassificationService",
  "version": "20200211173435_F31009",
  "created_at": "2020-02-12T01:35:00.448136Z",
  "env": {
    "conda_env": "name: bentoml-KerasTextClassificationService\nchannels:\n- defaults\

### Run 'predict' api with json data:

In [26]:
!KerasTextClassificationService run predict --input='{"text": "bad movie"}'

[2020-02-11 17:51:43,969] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will be default to same version as last PyPI release at version: 0.6.2. You can also force bentoml to use a specific version for deploying your BentoService bundle, by setting the config 'core/bentoml_deploy_version' to a pinned version or your custom BentoML on github, e.g.:'bentoml_deploy_version = git+https://github.com/{username}/bentoml.git@{branch}'
[2020-02-11 17:51:43,977] WARNING - Saved BentoService bundle version mismatch: loading BentoServie bundle create with BentoML version 0.6.2,  but loading from BentoML version 0.6.2+5.g3cbab90
[2020-02-11 17:51:43,979] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will be

## Build docker container with BentoService

BentoService bundles generated `Dockerfile`. To build docker image, you simply navigate to BentoService directory and run `docker build` from there

In [47]:
!cd {saved_path} && docker build -t keras-text-classify .

Sending build context to Docker daemon   5.41MB
Step 1/12 : FROM continuumio/miniconda3:4.7.12
4.7.12: Pulling from continuumio/miniconda3
Digest: sha256:6c979670684d970f8ba934bf9b7bf42e77c30a22eb96af1f30a039b484719159
Status: Downloaded newer image for continuumio/miniconda3:4.7.12
 ---> 406f2b43ea59
Step 2/12 : ENTRYPOINT [ "/bin/bash", "-c" ]
 ---> Using cache
 ---> 28172be83c07
Step 3/12 : EXPOSE 5000
 ---> Using cache
 ---> 840844d191d4
Step 4/12 : RUN set -x      && apt-get update      && apt-get install --no-install-recommends --no-install-suggests -y libpq-dev build-essential      && rm -rf /var/lib/apt/lists/*
 ---> Using cache
 ---> 243c05e712f3
Step 5/12 : RUN conda install pip numpy scipy       && pip install gunicorn
 ---> Using cache
 ---> 8fab95ab34fc
Step 6/12 : COPY . /bento
 ---> Using cache
 ---> 7a31febe4214
Step 7/12 : WORKDIR /bento
 ---> Using cache
 ---> 6aa52702e48c
Step 8/12 : RUN if [ -f /bento/setup.sh ]; then /bin/bash -c /bento/setup.sh; fi
 ---> Using cac

  Created wheel for BentoML: filename=BentoML-0.6.2+5.g3cbab90-py3-none-any.whl size=507025 sha256=92061051de863225bb0535e25985b467ba7b5b19699b3ef3826465aac5026344
  Stored in directory: /root/.cache/pip/wheels/6b/81/48/1d80a33960a7af644fec71f930327983b3f2e079d1101e14a8
Successfully built BentoML
  Attempting uninstall: BentoML
    Found existing installation: BentoML 0.6.2
    Uninstalling BentoML-0.6.2:
      Successfully uninstalled BentoML-0.6.2
Removing intermediate container a84cc2ab7940
 ---> 6eb0b7ca32d8
Step 12/12 : CMD ["bentoml serve-gunicorn /bento"]
 ---> Running in 825e35bc42e2
Removing intermediate container 825e35bc42e2
 ---> 91aa72136df5
Successfully built 91aa72136df5
Successfully tagged keras-text-classify:latest


In [48]:
!docker run -p 5000:5000 keras-text-classify

[2020-02-12 19:57:53,686] INFO - get_gunicorn_num_of_workers: 3, calculated by cpu count
[2020-02-12 19:57:54 +0000] [1] [INFO] Starting gunicorn 20.0.4
[2020-02-12 19:57:54 +0000] [1] [INFO] Listening at: http://0.0.0.0:5000 (1)
[2020-02-12 19:57:54 +0000] [1] [INFO] Using worker: sync
[2020-02-12 19:57:54 +0000] [8] [INFO] Booting worker with pid: 8
[2020-02-12 19:57:54 +0000] [9] [INFO] Booting worker with pid: 9
[2020-02-12 19:57:54 +0000] [10] [INFO] Booting worker with pid: 10
[2020-02-12 19:57:54,287] WARNING - Saved BentoService bundle version mismatch: loading BentoServie bundle create with BentoML version 0.6.2,  but loading from BentoML version 0.6.2+5.g3cbab90
[2020-02-12 19:57:54,411] WARNING - Saved BentoService bundle version mismatch: loading BentoServie bundle create with BentoML version 0.6.2,  but loading from BentoML version 0.6.2+5.g3cbab90
[2020-02-12 19:57:54,451] WARNING - Saved BentoService bundle version mismatch: loading BentoServie bundle create with BentoML

2020-02-12 19:57:58.191883: W tensorflow/compiler/jit/mark_for_compilation_pass.cc:1412] (One-time warning): Not using XLA:CPU for cluster because envvar TF_XLA_FLAGS=--tf_xla_cpu_global_jit was not set.  If you want XLA:CPU, either set that envvar, or use experimental_jit_scope to enable XLA:CPU.  To confirm that XLA is active, pass --vmodule=xla_compilation_cache=1 (as a proper command-line flag, not via TF_XLA_FLAGS) or set the envvar XLA_FLAGS=--xla_hlo_profile.
2020-02-12 19:57:58.191883: W tensorflow/compiler/jit/mark_for_compilation_pass.cc:1412] (One-time warning): Not using XLA:CPU for cluster because envvar TF_XLA_FLAGS=--tf_xla_cpu_global_jit was not set.  If you want XLA:CPU, either set that envvar, or use experimental_jit_scope to enable XLA:CPU.  To confirm that XLA is active, pass --vmodule=xla_compilation_cache=1 (as a proper command-line flag, not via TF_XLA_FLAGS) or set the envvar XLA_FLAGS=--xla_hlo_profile.
2020-02-12 19:57:58.218209: W tensorflow/compiler/jit/mark

## Deploy BentoService as REST API server to the cloud
BentoML support deployment to multiply cloud provider services, such as AWS Lambda, AWS Sagemaker, Google Cloudrun and etc. You can find the full list and guide on the documentation site at https://docs.bentoml.org/en/latest/deployment/index.html
For this project, we are going to deploy to AWS Sagemaker

**`bentoml sagemaker deploy` offers a single command to deployment**

In [55]:
!bentoml sagemaker deploy keras-text-classify -b KerasTextClassificationService:20200211173435_F31009 \
    --api-name predict --verbose

[2020-02-12 12:06:56,707] DEBUG - Using BentoML with local Yatai server
[2020-02-12 12:06:56,802] DEBUG - Upgrading tables to the latest revision
Deploying Sagemaker deployment |[2020-02-12 12:06:57,890] DEBUG - Created temporary directory: /private/var/folders/kn/xnc9k74x03567n1mx2tfqnpr0000gn/T/bentoml-temp-kdbp6qz8
-[2020-02-12 12:06:58,120] DEBUG - Getting docker login info from AWS
[2020-02-12 12:06:58,120] DEBUG - Building docker image: 192023623294.dkr.ecr.us-west-2.amazonaws.com/kerastextclassificationservice-sagemaker:20200211173435_F31009
\[2020-02-12 12:06:58,765] INFO - Step 1/11 : FROM continuumio/miniconda3:4.7.12
[2020-02-12 12:06:58,765] INFO - 

-[2020-02-12 12:06:59,697] INFO -  ---> 406f2b43ea59

[2020-02-12 12:06:59,697] INFO - Step 2/11 : EXPOSE 8080
[2020-02-12 12:06:59,697] INFO - 

[2020-02-12 12:06:59,698] INFO -  ---> Using cache

[2020-02-12 12:06:59,698] INFO -  ---> 7c8096d6922f

[2020-02-12 12:06:59,698] INFO - Step 3/11 : RUN set -x      && apt-get upd

/[2020-02-12 12:07:39,149] INFO - Collecting websocket-client>=0.32.0

[2020-02-12 12:07:39,165] INFO -   Downloading websocket_client-0.57.0-py2.py3-none-any.whl (200 kB)

\[2020-02-12 12:07:39,360] INFO - Collecting pyparsing>=2.0.2

[2020-02-12 12:07:39,372] INFO -   Downloading pyparsing-2.4.6-py2.py3-none-any.whl (67 kB)

[2020-02-12 12:07:39,410] INFO - Requirement already satisfied: setuptools in /opt/conda/lib/python3.7/site-packages (from cerberus->bentoml==0.6.2->-r /opt/program/requirements.txt (line 1)) (41.4.0)

[2020-02-12 12:07:39,415] INFO - Requirement already satisfied: urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1 in /opt/conda/lib/python3.7/site-packages (from requests->bentoml==0.6.2->-r /opt/program/requirements.txt (line 1)) (1.24.2)

[2020-02-12 12:07:39,422] INFO - Requirement already satisfied: chardet<3.1.0,>=3.0.2 in /opt/conda/lib/python3.7/site-packages (from requests->bentoml==0.6.2->-r /opt/program/requirements.txt (line 1)) (3.0.4)

[2020-02-12 12:07:39,424

\[2020-02-12 12:07:49,626] INFO - Installing collected packages: prometheus-client, grpcio, ruamel.yaml.clib, ruamel.yaml, python-json-logger, configparser, sqlalchemy, websocket-client, docker, pyparsing, packaging, cerberus, python-dateutil, protobuf, tabulate, itsdangerous, Werkzeug, MarkupSafe, Jinja2, click, flask, humanfriendly, jmespath, docutils, botocore, s3transfer, boto3, pytz, pandas, Mako, python-editor, alembic, bentoml, tensorflow-estimator, markdown, absl-py, tensorboard, keras-preprocessing, astor, h5py, keras-applications, wrapt, gast, google-pasta, termcolor, tensorflow

|[2020-02-12 12:08:13,342] INFO - Successfully installed Jinja2-2.11.1 Mako-1.1.1 MarkupSafe-1.1.1 Werkzeug-1.0.0 absl-py-0.9.0 alembic-1.4.0 astor-0.8.1 bentoml-0.6.2 boto3-1.11.15 botocore-1.14.15 cerberus-1.3.2 click-7.0 configparser-4.0.2 docker-4.2.0 docutils-0.15.2 flask-1.1.1 gast-0.3.3 google-pasta-0.1.8 grpcio-1.27.1 h5py-2.10.0 humanfriendly-6.1 itsdangerous-1.1.0 jmespath-0.9.4 keras-app

|[2020-02-12 12:10:10,091] DEBUG - AWS describe endpoint response: {'EndpointName': 'bobo-keras-text-classify', 'EndpointArn': 'arn:aws:sagemaker:us-west-2:192023623294:endpoint/bobo-keras-text-classify', 'EndpointConfigName': 'bobo-keras-text-c-KerasTextClassificat-20200211173435-F31009', 'EndpointStatus': 'Creating', 'CreationTime': datetime.datetime(2020, 2, 12, 12, 9, 43, 287000, tzinfo=tzlocal()), 'LastModifiedTime': datetime.datetime(2020, 2, 12, 12, 9, 43, 287000, tzinfo=tzlocal()), 'ResponseMetadata': {'RequestId': '161a4c03-188a-41cd-992f-c6ef7c4234a9', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '161a4c03-188a-41cd-992f-c6ef7c4234a9', 'content-type': 'application/x-amz-json-1.1', 'content-length': '314', 'date': 'Wed, 12 Feb 2020 20:10:10 GMT'}, 'RetryAttempts': 0}}
/[2020-02-12 12:10:15,354] DEBUG - AWS describe endpoint response: {'EndpointName': 'bobo-keras-text-classify', 'EndpointArn': 'arn:aws:sagemaker:us-west-2:192023623294:endpoint/bobo-keras-text-cl

/[2020-02-12 12:11:08,895] DEBUG - AWS describe endpoint response: {'EndpointName': 'bobo-keras-text-classify', 'EndpointArn': 'arn:aws:sagemaker:us-west-2:192023623294:endpoint/bobo-keras-text-classify', 'EndpointConfigName': 'bobo-keras-text-c-KerasTextClassificat-20200211173435-F31009', 'EndpointStatus': 'Creating', 'CreationTime': datetime.datetime(2020, 2, 12, 12, 9, 43, 287000, tzinfo=tzlocal()), 'LastModifiedTime': datetime.datetime(2020, 2, 12, 12, 9, 43, 287000, tzinfo=tzlocal()), 'ResponseMetadata': {'RequestId': 'd12bdf14-8c57-4754-a833-4e4ce5cb0c0d', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'd12bdf14-8c57-4754-a833-4e4ce5cb0c0d', 'content-type': 'application/x-amz-json-1.1', 'content-length': '314', 'date': 'Wed, 12 Feb 2020 20:11:08 GMT'}, 'RetryAttempts': 0}}
-[2020-02-12 12:11:14,165] DEBUG - AWS describe endpoint response: {'EndpointName': 'bobo-keras-text-classify', 'EndpointArn': 'arn:aws:sagemaker:us-west-2:192023623294:endpoint/bobo-keras-text-cl

...

\[2020-02-12 12:14:00,573] DEBUG - AWS describe endpoint response: {'EndpointName': 'bobo-keras-text-classify', 'EndpointArn': 'arn:aws:sagemaker:us-west-2:192023623294:endpoint/bobo-keras-text-classify', 'EndpointConfigName': 'bobo-keras-text-c-KerasTextClassificat-20200211173435-F31009', 'EndpointStatus': 'Creating', 'CreationTime': datetime.datetime(2020, 2, 12, 12, 9, 43, 287000, tzinfo=tzlocal()), 'LastModifiedTime': datetime.datetime(2020, 2, 12, 12, 9, 43, 287000, tzinfo=tzlocal()), 'ResponseMetadata': {'RequestId': '18a08516-a21f-4cee-9f8c-e96583159e9b', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '18a08516-a21f-4cee-9f8c-e96583159e9b', 'content-type': 'application/x-amz-json-1.1', 'content-length': '314', 'date': 'Wed, 12 Feb 2020 20:14:00 GMT'}, 'RetryAttempts': 0}}
|[2020-02-12 12:14:05,757] DEBUG - AWS describe endpoint response: {'EndpointName': 'bobo-keras-text-classify', 'EndpointArn': 'arn:aws:sagemaker:us-west-2:192023623294:endpoint/bobo-keras-text-cl

|[2020-02-12 12:14:57,717] DEBUG - AWS describe endpoint response: {'EndpointName': 'bobo-keras-text-classify', 'EndpointArn': 'arn:aws:sagemaker:us-west-2:192023623294:endpoint/bobo-keras-text-classify', 'EndpointConfigName': 'bobo-keras-text-c-KerasTextClassificat-20200211173435-F31009', 'EndpointStatus': 'Creating', 'CreationTime': datetime.datetime(2020, 2, 12, 12, 9, 43, 287000, tzinfo=tzlocal()), 'LastModifiedTime': datetime.datetime(2020, 2, 12, 12, 9, 43, 287000, tzinfo=tzlocal()), 'ResponseMetadata': {'RequestId': '0cd6cc89-9709-4851-95be-c5ff1ac1ce2c', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '0cd6cc89-9709-4851-95be-c5ff1ac1ce2c', 'content-type': 'application/x-amz-json-1.1', 'content-length': '314', 'date': 'Wed, 12 Feb 2020 20:14:57 GMT'}, 'RetryAttempts': 0}}
/[2020-02-12 12:15:02,904] DEBUG - AWS describe endpoint response: {'EndpointName': 'bobo-keras-text-classify', 'EndpointArn': 'arn:aws:sagemaker:us-west-2:192023623294:endpoint/bobo-keras-text-cl

/[2020-02-12 12:15:55,014] DEBUG - AWS describe endpoint response: {'EndpointName': 'bobo-keras-text-classify', 'EndpointArn': 'arn:aws:sagemaker:us-west-2:192023623294:endpoint/bobo-keras-text-classify', 'EndpointConfigName': 'bobo-keras-text-c-KerasTextClassificat-20200211173435-F31009', 'EndpointStatus': 'Creating', 'CreationTime': datetime.datetime(2020, 2, 12, 12, 9, 43, 287000, tzinfo=tzlocal()), 'LastModifiedTime': datetime.datetime(2020, 2, 12, 12, 9, 43, 287000, tzinfo=tzlocal()), 'ResponseMetadata': {'RequestId': '4d163315-1586-461a-93bd-d77229d0e70e', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '4d163315-1586-461a-93bd-d77229d0e70e', 'content-type': 'application/x-amz-json-1.1', 'content-length': '314', 'date': 'Wed, 12 Feb 2020 20:15:54 GMT'}, 'RetryAttempts': 0}}
\[2020-02-12 12:16:00,211] DEBUG - AWS describe endpoint response: {'EndpointName': 'bobo-keras-text-classify', 'EndpointArn': 'arn:aws:sagemaker:us-west-2:192023623294:endpoint/bobo-keras-text-cl

-[2020-02-12 12:16:52,142] DEBUG - AWS describe endpoint response: {'EndpointName': 'bobo-keras-text-classify', 'EndpointArn': 'arn:aws:sagemaker:us-west-2:192023623294:endpoint/bobo-keras-text-classify', 'EndpointConfigName': 'bobo-keras-text-c-KerasTextClassificat-20200211173435-F31009', 'EndpointStatus': 'Creating', 'CreationTime': datetime.datetime(2020, 2, 12, 12, 9, 43, 287000, tzinfo=tzlocal()), 'LastModifiedTime': datetime.datetime(2020, 2, 12, 12, 9, 43, 287000, tzinfo=tzlocal()), 'ResponseMetadata': {'RequestId': '7df585d0-2b09-40f4-8231-4edebe3be029', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '7df585d0-2b09-40f4-8231-4edebe3be029', 'content-type': 'application/x-amz-json-1.1', 'content-length': '314', 'date': 'Wed, 12 Feb 2020 20:16:51 GMT'}, 'RetryAttempts': 0}}
\[2020-02-12 12:16:57,324] DEBUG - AWS describe endpoint response: {'EndpointName': 'bobo-keras-text-classify', 'EndpointArn': 'arn:aws:sagemaker:us-west-2:192023623294:endpoint/bobo-keras-text-cl

-[2020-02-12 12:17:49,750] DEBUG - AWS describe endpoint response: {'EndpointName': 'bobo-keras-text-classify', 'EndpointArn': 'arn:aws:sagemaker:us-west-2:192023623294:endpoint/bobo-keras-text-classify', 'EndpointConfigName': 'bobo-keras-text-c-KerasTextClassificat-20200211173435-F31009', 'EndpointStatus': 'Creating', 'CreationTime': datetime.datetime(2020, 2, 12, 12, 9, 43, 287000, tzinfo=tzlocal()), 'LastModifiedTime': datetime.datetime(2020, 2, 12, 12, 9, 43, 287000, tzinfo=tzlocal()), 'ResponseMetadata': {'RequestId': '901ae4a0-e8ad-4899-93f6-e540f7cde058', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '901ae4a0-e8ad-4899-93f6-e540f7cde058', 'content-type': 'application/x-amz-json-1.1', 'content-length': '314', 'date': 'Wed, 12 Feb 2020 20:17:49 GMT'}, 'RetryAttempts': 0}}
|[2020-02-12 12:17:54,922] DEBUG - AWS describe endpoint response: {'EndpointName': 'bobo-keras-text-classify', 'EndpointArn': 'arn:aws:sagemaker:us-west-2:192023623294:endpoint/bobo-keras-text-cl

Use `bentoml sagemaker list` to display all sagemaker deployments with BentoML

In [ ]:
!bentoml sagemaker list

To get the latest deployment state and information, use `bentoml sagemaker get` command

In [57]:
!bentoml sagemaker get keras-text-classify

{
  "namespace": "bobo",
  "name": "keras-text-classify",
  "spec": {
    "bentoName": "KerasTextClassificationService",
    "bentoVersion": "20200211173435_F31009",
    "operator": "AWS_SAGEMAKER",
    "sagemakerOperatorConfig": {
      "region": "us-west-2",
      "instanceType": "ml.m4.xlarge",
      "instanceCount": 1,
      "apiName": "predict"
    }
  },
  "state": {
    "state": "RUNNING",
    "infoJson": {
      "EndpointName": "bobo-keras-text-classify",
      "EndpointArn": "arn:aws:sagemaker:us-west-2:192023623294:endpoint/bobo-keras-text-classify",
      "EndpointConfigName": "bobo-keras-text-c-KerasTextClassificat-20200211173435-F31009",
      "ProductionVariants": [
        {
          "VariantName": "bobo-keras-text-c-KerasTextClassificat-20200211173435-F31009",
          "DeployedImages": [
            {
              "SpecifiedImage": "192023623294.dkr.ecr.us-west-2.amazonaws.com/kerastextclassificationservice-sagemaker:20200211173435_F31009",
              "ResolvedIm

We will use AWS CLI tool to test the deployment with sample data

In [58]:
!aws sagemaker-runtime invoke-endpoint --endpoint-name bobo-keras-text-classify \
--body '{"text": "best movie ever"}' --content-type application/json output.json && cat output.json

{
    "ContentType": "application/json",
    "InvokedProductionVariant": "bobo-keras-text-c-KerasTextClassificat-20200211173435-F31009"
}
[[1]]

`bentoml sagemaker delete` will delete the sagemaker deployment and related resources

In [59]:
!bentoml sagemaker delete keras-text-classify

Successfully deleted AWS Sagemaker deployment "keras-text-classify"
